In [57]:
import os
import json
import numpy as np
import collections
from Bio import SeqIO

datadict = {}

datadict = collections.defaultdict(dict)
for record in SeqIO.parse("../disprot_OK_fullset.fasta", "fasta"):
    uniprot_id = record.id
    seq = str(record.seq)
    datadict[uniprot_id]['seq'] = seq

for record in SeqIO.parse("../disprot_OK_fullset_annotations.fasta", "fasta"):
    uniprot_id = record.id
    seq = str(record.seq)
    datadict[uniprot_id]['disorder'] = seq

print(f"Loaded {len(datadict.keys())} proteins")

Loaded 2145 proteins


In [58]:
disprot_annot_file = "/biodata/franco/datasets/disprot/DisProt_release_2023_12_with_ambiguous_evidences_consensus_regions.fasta"
disprot_json_file = "/biodata/franco/datasets/disprot/DisProt_release_2023_12_with_ambiguous_evidences.json"

with open(disprot_json_file) as f:
    new_disprot_data = json.load(f)

new_datadict = {}
new_datadict = collections.defaultdict(dict)
for record in SeqIO.parse(disprot_annot_file, "fasta"):
    acc = record.description.split(" ")[1].split("=")[1]
    seq = str(record.seq)
    new_datadict[acc]['disorder'] = seq

In [59]:
new_disprot_data['data'][0].keys()
print(new_disprot_data['data'][0]['disprot_id'])
print(new_disprot_data['data'][0]['acc'])
print(new_disprot_data['data'][0]['length'])
print(new_disprot_data['data'][0]['sequence'])

for protein in new_disprot_data['data']:
    acc = protein['acc']
    if acc in datadict:
        new_datadict[acc]['seq'] = protein['sequence']

DP00003
P03265
529
MASREEEQRETTPERGRGAARRPPTMEDVSSPSPSPPPPRAPPKKRMRRRIESEDEEDSSQDALVPRTPSPRPSTSAADLAIAPKKKKKRPSPKPERPPSPEVIVDSEEEREDVALQMVGFSNPPVLIKHGKGGKRTVRRLNEDDPVARGMRTQEEEEEPSEAESEITVMNPLSVPIVSAWEKGMEAARALMDKYHVDNDLKANFKLLPDQVEALAAVCKTWLNEEHRGLQLTFTSKKTFVTMMGRFLQAYLQSFAEVTYKHHEPTGCALWLHRCAEIEGELKCLHGSIMINKEHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNTDARCCVHDAACPANQFSGKSCGMFFSEGAKAQVAFKQIKAFMQALYPNAQTGHGHLLMPLRCECNSKPGHAPFLGRQLPKLTPFALSNAEDLDADLISDKSVLASVHHPALIVFQCCNPVYRNSRAQGGGPNCDFKISAPDLLNALVMVRSLWSENFTELPRMVVPEFKWSTKHQYRNVSLPVAHSDARQNPFDF


In [67]:
new_datadict['P03265']

{'disorder': '-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD-----------------------------------------------------------------------------------------------------------------------DDDDDDDDDDD-----------------------------------------------------------------',
 'seq': 'MASREEEQRETTPERGRGAARRPPTMEDVSSPSPSPPPPRAPPKKRMRRRIESEDEEDSSQDALVPRTPSPRPSTSAADLAIAPKKKKKRPSPKPERPPSPEVIVDSEEEREDVALQMVGFSNPPVLIKHGKGGKRTVRRLNEDDPVARGMRTQEEEEEPSEAESEITVMNPLSVPIVSAWEKGMEAARALMDKYHVDNDLKANFKLLPDQVEALAAVCKTWLNEEHRGLQLTFTSKKTFVTMMGRFLQAYLQSFAEVTYKHHEPTGCALWLHRCAEIEGELKCLHGSIMINKEHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNTDARCCVHDAACPANQFSGKSCGMFFSEGAKAQVAFKQIKAFMQALYPNAQTGHGHLLMPLRCECNSKPGHAPFLGRQLPKLTPFALSNAEDLDADLISDKSVLASVHHPAL

In [76]:
disodiff = 0
disofails = 0
lenfails = 0
seqfails = 0
same = 0
ok_uniprots = []
for uniprot_id in datadict:
    if uniprot_id in new_datadict:
        if datadict[uniprot_id]['seq'] == new_datadict[uniprot_id]['seq']:
            if datadict[uniprot_id]['disorder'] != new_datadict[uniprot_id]['disorder']:
                disodiff += 1
                if len(datadict[uniprot_id]['disorder']) == len(new_datadict[uniprot_id]['disorder']):
                    #print(f"Lengths differ: {len(datadict[uniprot_id]['disorder'])} vs {len(new_datadict[uniprot_id]['disorder'])}")
                    ok_uniprots.append(uniprot_id)
                else:
                    lenfails += 1
            else:
                ok_uniprots.append(uniprot_id)
                same += 1
        else:
            # print(f"Uniprot {uniprot_id} seq is different")
            # print(f"Seq1: {datadict[uniprot_id]['seq']}")
            # print(f"Seq2: {new_datadict[uniprot_id]['seq']}")
            seqfails += 1
    else:
        disofails += 1

if lenfails != 0:
    raise

if lenfails == 0:
    print(f"OK. {len(ok_uniprots)} uniprots. {same} same.")
    print(f"{disofails} fails. {disodiff} differences. {lenfails} length differences. {seqfails} seq differences.")
else:
    print(f"{disofails} fails. {disodiff} differences. {lenfails} length differences. {seqfails} seq differences.")
    raise


OK. 2092 uniprots. 1905 same.
47 fails. 187 differences. 0 length differences. 6 seq differences.


In [78]:

# export new annotations as fasta for all previous uniprots
with open(f"../disprot_OK_fullset_annotations_12_2023.fasta", "w") as f:
    with open(f"../disprot_OK_fullset_12_2023.fasta", "w") as f2:
        for uniprot_id in ok_uniprots:
            if uniprot_id in new_datadict:
                f.write(f">{uniprot_id}\n{new_datadict[uniprot_id]['disorder']}\n")
                f2.write(f">{uniprot_id}\n{datadict[uniprot_id]['seq']}\n")